<a href="https://colab.research.google.com/github/HwangHanJae/crawling_pratice/blob/main/crawling_fs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

크롤링 라이브러리 다운로드

In [2]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver

     |████████████████████████████████| 968 kB 7.9 MB/s 
     |████████████████████████████████| 138 kB 53.3 MB/s 
     |████████████████████████████████| 359 kB 38.2 MB/s 
     |████████████████████████████████| 55 kB 3.7 MB/s 
     |████████████████████████████████| 3.6 MB 57.0 MB/s 
     |████████████████████████████████| 58 kB 5.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.down

In [184]:
#라이브러리 로드
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import requests
import warnings
warnings.filterwarnings('ignore')

In [185]:
#테마별 url을 입력시에 종목별 코드를 찾는 함수
def find_code(url):
  link = requests.get(url)
  soup = BeautifulSoup(link.content, 'html.parser')
  dic = {}
  for data in soup.find_all("div",{"class":"name_area"}):
    name = data.find('a').text.strip()
    code = data.find('a')['href'][-6:].strip()
    dic[name]=code
  return dic

In [186]:
#코드를 입력시에 2021/09년도 매출액, 자본총계, eps(원) 데이터 추출 후 데이터프레임 생성
def create_df(code):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--single-process")
    chrome_options.add_argument("--disable-dev-shm-usage")

    url = 'https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd={}'.format(code)
    driver = webdriver.Chrome(chrome_options=chrome_options)
    driver.get(url)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.select('table')
    table_html = str(table)
    fs = pd.read_html(table_html)[12]
    df = fs.T

    columns_name = df.iloc[0]
    df = df.iloc[1:]
    df.columns = columns_name
    df = df.reset_index(drop=False).iloc[:,1:].rename(columns={"level_1":"분기"})
    df = df.loc[df['분기']=='2021/09  (IFRS연결)'][["매출액","자본총계","EPS(원)"]]
    return df

코드 실행

In [187]:
ma_url = "https://finance.naver.com/sise/sise_group_detail.nhn?type=theme&no=408"
ship_list = "https://finance.naver.com/sise/sise_group_detail.nhn?type=theme&no=36"

ma_dic = find_code(ma_url)
ship_dic = find_code(ship_list)

print(ma_dic)
print(ship_dic)

{'에스엘바이오닉스': '214310', '우리바이오': '082850', '엔에프씨': '265740', '오성첨단소재': '052420', '애머릿지': '900100', '화일약품': '061250', '노터스': '278650', 'EDGC': '245620', '아이큐어': '175250', '넥스트BT': '065170', '한국비엔씨': '256840', '마이더스AI': '222810'}
{'팬오션': '028670', 'KSS해운': '044450', '대한해운': '005880', '흥아해운': '003280', 'HMM': '011200'}


In [188]:
dfs = []
#마리화나(대마) df 만들기
for name in ma_dic:
  df = create_df(ma_dic[name])
  df["종목명"] = name
  dfs.append(df)
ma_df = pd.concat(dfs)
ma_df.columns.rename('', inplace=True)
ma_df = ma_df.reset_index(drop=True)

In [189]:
ma_df

,매출액,자본총계,EPS(원),종목명
0,54.0,452.0,282.0,에스엘바이오닉스
1,3929.0,1914.0,219.0,우리바이오
2,110.0,546.0,288.0,엔에프씨
3,243.0,2168.0,16.0,오성첨단소재
4,177.0,551.0,25.0,애머릿지
5,154.0,618.0,187.0,노터스
6,243.0,664.0,9.0,EDGC
7,144.0,943.0,-530.0,아이큐어
8,193.0,817.0,-12.0,넥스트BT
9,203.0,383.0,10.0,마이더스AI


In [190]:
dfs = []
for name in ship_dic:
  df = create_df(ship_dic[name])
  df["종목명"] = name
  dfs.append(df)
ship_df = pd.concat(dfs)
ship_df.columns.rename('', inplace=True)
ship_df = ship_df.reset_index(drop=True)

In [191]:
ship_df

,매출액,자본총계,EPS(원),종목명
0,13282.0,33672.0,335.0,팬오션
1,867.0,3322.0,957.0,KSS해운
2,3097.0,13563.0,379.0,대한해운
3,193.0,1236.0,3.0,흥아해운
4,40164.0,77259.0,5673.0,HMM


In [195]:
df = pd.concat([ma_df,ship_df])
df = df.reset_index(drop=True)

In [196]:
df

,매출액,자본총계,EPS(원),종목명
0,54.0,452.0,282.0,에스엘바이오닉스
1,3929.0,1914.0,219.0,우리바이오
2,110.0,546.0,288.0,엔에프씨
3,243.0,2168.0,16.0,오성첨단소재
4,177.0,551.0,25.0,애머릿지
5,154.0,618.0,187.0,노터스
6,243.0,664.0,9.0,EDGC
7,144.0,943.0,-530.0,아이큐어
8,193.0,817.0,-12.0,넥스트BT
9,203.0,383.0,10.0,마이더스AI
